In [1]:
#set to 1 if to import shapely and use it, without, some time zone retrievals can not be made
#the rest of the packages are required for time zone fetching
#can be set through initiallization notebook
shapely_f = 1

In [2]:
#[pygecoders, geopy, pycountry, pytz, tzwhere, shapley] are the uncommon packages used
#besides shapely all are a requirement
import os
import pandas as p
import time
import numpy as np
from datetime import datetime, timedelta
from geopy.geocoders import Nominatim
from pygeocoder import Geocoder
import pycountry
from pytz import timezone
import pytz
from tzwhere import tzwhere
if shapely_f:
    import shapely

In [3]:
#path directory of tables

path = os.getcwd()
path = path[:-path[::-1].index('/')]
path_b = path + 'source_tables/built/'

In [4]:
events = p.read_csv(path_b + 'events.csv', usecols = [0,2,4], dtype={'display_id': int, 'timestamp' : int, 'geo_location' : str})

#remove rows where geo_location is null, keep events where missing locations will be offset of 0
events_parsing = events[events.geo_location.notnull()]

In [5]:
print events_parsing.head()

   display_id  timestamp geo_location
0           1         61    US>SC>519
1           2         81    US>CA>807
2           3        182    US>MI>505
3           4        234    US>WV>564
4           5        338        SG>00


In [6]:
#functions for parsing the abbreviations from the geo_location
def parse_country(x):
    if type(x) == float: # for nans
        return None
    x = x[:2]
    if x.isdigit():
        return None
    if x == '--':
        return None
    else:
        return x
    
def parse_state(x):
    if type(x) == float: # for nans
        return None
    x = x[:2] + '-' + x[3:5]
    if len(x) < 4:
        return None
    if x.isdigit():
        return None
    if x == '--':
        return None
    if x == '':
        return None
    else:
        return x
    
def nan_check(x):
    if x == -1:
        return True
    if type(x) == None:
        return True
    if x != x:
        return True
    return False

In [7]:
#dictionaries to get full country and state names
country_mapping = {country.alpha_2 : country.name for country in pycountry.countries}
state_mapping = {country_state.code : country_state.name for country_state in pycountry.subdivisions}

#were missing so added manualy
country_mapping['AN'] = 'Netherlands'
country_mapping['FX'] = 'France'

In [8]:
#get a table of all country - state options ready to pull timezones
#-1 resembles null (replacing for merging purposes)
events_parsing = events_parsing.assign(country = events_parsing['geo_location'].apply(parse_country))
events_parsing = events_parsing.assign(state = events_parsing['geo_location'].apply(parse_state))
events_parsing = events_parsing.assign(country = events_parsing['country'].map(country_mapping))
events_parsing = events_parsing.assign(state = events_parsing['state'].map(state_mapping))
events_parsing = events_parsing[events_parsing.country.notnull()]
events_parsing = events_parsing.fillna(-1)

In [9]:
print events_parsing.head()

   display_id  timestamp geo_location        country           state
0           1         61    US>SC>519  United States  South Carolina
1           2         81    US>CA>807  United States      California
2           3        182    US>MI>505  United States        Michigan
3           4        234    US>WV>564  United States   West Virginia
4           5        338        SG>00      Singapore              -1


In [10]:
#new table of country and state
country_state_combos = events_parsing[['country','state']].groupby(['country','state']).count().reset_index()

In [11]:
print country_state_combos.head()
print country_state_combos.shape

       country   state
0  Afghanistan      -1
1      Albania      -1
2      Algeria      -1
3      Algeria   Adrar
4      Algeria  Biskra
(648, 2)


In [12]:
#pull timezones for each country - state pair
#theres' a sleep call every time host cancels connection
gn = Nominatim()

if shapely_f:
    tz = tzwhere.tzwhere(shapely=True, forceTZ=True)
else:
    tz = tzwhere.tzwhere()

total_len = country_state_combos.shape[0]

for i, row in enumerate(country_state_combos.itertuples()):
    if i % 36 == 0:
        print repr(float(i) / total_len)[:4] + '% done'
    nulled = 0
    if nan_check(row.state):
        place = row.country
    else:
        place = row.country + ' ' + row.state
    #get the location details
    set = 0
    while(set == 0):
        try:
            location = gn.geocode(place)
            if location == None:
                place = row.country
                location = gn.geocode(place)
                if location == None:
                    nulled = 1
                    print repr(i) + ' location null'
            set = 1
        except:
            time.sleep(5)
    if nulled == 1:
        continue
    #get the time zone details of the location
    if shapely_f:
        timezone_str = tz.tzNameAt(location.latitude, location.longitude, forceTZ=True)
    else:
        timezone_str = tz.tzNameAt(location.latitude, location.longitude)
    if timezone_str == None:
        set = 0
        while(set == 0):
            try:
                location = gn.geocode(row.country)
                if location == None:
                    nulled = 1
                    print repr(i) + ' location null'
                set = 1
            except:
                time.sleep(5)
        if nulled == 1:
            continue
        if shapely_f:
            timezone_str = tz.tzNameAt(location.latitude, location.longitude, forceTZ=True)
        else:
            timezone_str = tz.tzNameAt(location.latitude, location.longitude)
        if timezone_str == None:
            print repr(i) + ' timezone null'
            continue
    if timezone_str == 'uninhabited':
        print repr(i) + ' uninhabited'
        continue

    #get the offset from UTC (offset 0)
    pyt = timezone(timezone_str).localize(datetime(2016, 8, 1, 0, 00, 00))

    #set the offset in the table
    country_state_combos.set_value(i, 'offset', pyt.utcoffset().total_seconds())
print "done"

0.0% done
0.05% done
0.11% done
99 uninhabited
0.16% done
0.22% done
0.27% done
197 location null
0.33% done
0.38% done
267 uninhabited
0.44% done
316 timezone null
323 timezone null
0.5% done
0.55% done
362 uninhabited
0.61% done
0.66% done
0.72% done
0.77% done
0.83% done
0.88% done
0.94% done
636 location null
done


In [13]:
#the only nulls if shapley is present, added manualy
country_state_combos = country_state_combos.set_value(99, 'offset', -10*60*60)
country_state_combos = country_state_combos.set_value(197, 'offset', 60*60)
country_state_combos = country_state_combos.set_value(267, 'offset', 13*60*60)
country_state_combos = country_state_combos.set_value(316, 'offset', 12*60*60)
country_state_combos = country_state_combos.set_value(323, 'offset', 11*60*60)
country_state_combos = country_state_combos.set_value(362, 'offset', 9*60*60)
country_state_combos = country_state_combos.set_value(636, 'offset', -4*60*60)

In [14]:
#without shapely, many nulls exist, fill with offset 0
country_state_combos = country_state_combos.fillna(0)

,country,state,offset
0,Afghanistan,-1,16200.0
1,Albania,-1,7200.0
2,Algeria,-1,3600.0
3,Algeria,Adrar,3600.0
4,Algeria,Biskra,3600.0
5,Algeria,Blida,3600.0
6,Algeria,Guelma,3600.0
7,Algeria,Laghouat,3600.0
8,Algeria,Médéa,3600.0
9,Algeria,Oran,3600.0


In [15]:
country_state_combos.head()

,country,state,offset
0,Afghanistan,-1,16200.0
1,Albania,-1,7200.0
2,Algeria,-1,3600.0
3,Algeria,Adrar,3600.0
4,Algeria,Biskra,3600.0


In [16]:
events_parsing = events_parsing.merge(country_state_combos, how='left', on=['country','state']).drop(events_parsing.columns[[2,3,4]], axis=1)

In [17]:
print events_parsing.head()
print events.head()

   display_id  timestamp   offset
0           1         61 -14400.0
1           2         81 -25200.0
2           3        182 -14400.0
3           4        234 -14400.0
4           5        338  28800.0
   display_id  timestamp geo_location
0           1         61    US>SC>519
1           2         81    US>CA>807
2           3        182    US>MI>505
3           4        234    US>WV>564
4           5        338        SG>00


In [18]:
events = events.merge(events_parsing.drop('timestamp', axis = 1), how = 'left', on = 'display_id').drop('geo_location',axis = 1)
#some of the offsets are nulls since there are some corrupt geo_locations
events = events.fillna(0)

In [19]:
events.head(10)

,display_id,timestamp,offset
0,1,61,-14400.0
1,2,81,-25200.0
2,3,182,-14400.0
3,4,234,-14400.0
4,5,338,28800.0
5,6,395,-14400.0
6,7,602,-21600.0
7,8,638,-14400.0
8,9,667,-14400.0
9,10,693,-18000.0


In [20]:
#add offset to timestamp as miliseconds
events.timestamp = events.timestamp + (events.offset * 1000)

#if timestamp is negative add a weeks' time
events.loc[events['timestamp'] < 0, 'timestamp'] = events['timestamp'] + 1000*60*60*24*7

In [21]:
#converting timestamp field to hour and day
events['hour'] = (events.timestamp // (3600 * 1000)) % 24
events['day'] = events.timestamp // (3600 * 24 * 1000)

In [22]:
#adding weekend column
#diving day hours to 4 sections to combine results of adjacent hours
events['weekend'] = events['morning'] = events['noon'] = events['evening'] = events['night'] = 0
events.ix[events['day'].isin([4, 5, 11, 12]), 'weekend'] = 1
events.ix[(events['hour'] < 6) | (events['hour'] > 23), 'night'] = 1
events.ix[(events['hour'] >= 6) & (events['hour'] < 12), 'morning'] = 1
events.ix[(events['hour'] >= 12) & (events['hour'] < 18), 'noon'] = 1
events.ix[(events['hour'] >= 18) & (events['hour'] <= 23), 'evening'] = 1

In [23]:
events.drop(['timestamp', 'hour', 'day','offset'], axis=1, inplace=True)
print events.head(10)

   display_id  weekend  morning  noon  evening  night
0           1        0        0     0        1      0
1           2        0        0     1        0      0
2           3        0        0     0        1      0
3           4        0        0     0        1      0
4           5        0        1     0        0      0
5           6        0        0     0        1      0
6           7        0        0     0        1      0
7           8        0        0     0        1      0
8           9        0        0     0        1      0
9          10        0        0     0        1      0


In [24]:
events.to_csv(path_b + 'time_table.csv', index = False)